In [1]:
import os

os.chdir(r'/Users/jialunlyu/Documents/GitHub/ProjectBuffalo')

In [2]:
import pandas as pd

In [3]:
import buffalo.ingestion as ingestion

In [4]:
ingestion.configuration.Configuration.print()

{'storage_folder': '../__cached_data', 'api_keys': {<API.ADVANTAGE: 1>: '2rrNROO0beX90lPH7ixQOp0mT_9SwF0d'}, 'watched_tickers': ['IBM', 'MSFT']}


In [5]:
ingestor = ingestion.DataIngestion(ingestion.enum.API.ADVANTAGE)

In [6]:
tickers = ['AAPL', 'MSFT', 'IBM', 'META', 'JNJ', 'PFE', 'UNH', 'MARK', 'XLV', 'JPM', 'BAC', 'GS', 'MS', 'XLF', 'PG', 'KO', 'PEP', 'NKE', 'XLP', 'GE', 'HON', 'UTX', 'MMM', 'XLI']

#### Adjusted Daily Stock Data

In [7]:
daily_stocks_args = pd.DataFrame({
    'symbol': tickers,
    'interval': 'daily',
    'adjusted': True,
    'data_type': ingestion.enum.DataType.STOCK,
    'ingestion_type': ingestion.enum.IngestionType.REST,
    'data_name': 'ADJUSTED_DAILY_STOCK'
})

In [8]:
ingestor.ingestion_from_web(daily_stocks_args)

100%|██████████| 24/24 [04:10<00:00, 10.43s/it]


In [9]:
ingestor.store_data(r'cached_data/ingestion.sqlite', data_names=None)

#### Company Info

In [10]:
company_info = pd.DataFrame({
    'symbol': tickers,
    'data_type': ingestion.enum.DataType.COMPANY,
    'ingestion_type': ingestion.enum.IngestionType.REST
})

In [11]:
company_info = company_info.merge(pd.DataFrame({
    'function': ['OVERVIEW', 'INCOME_STATEMENT', 'BALANCE_SHEET', 'CASH_FLOW', 'EARNINGS', 'EARNINGS_CALENDAR'],
    'data_name': ['COMPANY_OVERVIEW', 'COMPANY_INCOME_STATEMENT', 'COMPANY_BALANCE_SHEET', 'COMPANY_CASH_FLOW', 'COMPANY_EARNINGS', 'COMPANY_EARNINGS_CALENDAR_3M']
    }), how='cross')

In [12]:
ingestor.ingestion_from_web(company_info)

  3%|▎         | 5/144 [00:01<00:26,  5.15it/s]/Users/jialunlyu/Documents/GitHub/ProjectBuffalo/buffalo/ingestion/advantage.py:73: UserWarning: Ingestion results from https://www.alphavantage.co/query?function=EARNINGS_CALENDAR&symbol=AAPL&horizon=3month&apikey=2rrNROO0beX90lPH7ixQOp0mT_9SwF0d in 0 rows.
  warnings.warn(f'Ingestion results from {url} in 0 rows.')
 37%|███▋      | 53/144 [08:10<07:42,  5.08s/it]/Users/jialunlyu/Documents/GitHub/ProjectBuffalo/buffalo/ingestion/advantage.py:73: UserWarning: Ingestion results from https://www.alphavantage.co/query?function=EARNINGS_CALENDAR&symbol=XLV&horizon=3month&apikey=2rrNROO0beX90lPH7ixQOp0mT_9SwF0d in 0 rows.
  warnings.warn(f'Ingestion results from {url} in 0 rows.')
 58%|█████▊    | 83/144 [13:16<05:09,  5.08s/it]/Users/jialunlyu/Documents/GitHub/ProjectBuffalo/buffalo/ingestion/advantage.py:73: UserWarning: Ingestion results from https://www.alphavantage.co/query?function=EARNINGS_CALENDAR&symbol=XLF&horizon=3month&apikey=2rrNRO

#### Listing Info

In [13]:
listing_info = pd.DataFrame({
    'date': ['2022-01-01'],
    'data_type': ingestion.enum.DataType.STOCK_LISTING,
    'ingestion_type': ingestion.enum.IngestionType.REST,
    'data_name': 'LISTING_INFO'
})

In [14]:
ingestor.ingestion_from_web(listing_info)

  0%|          | 0/1 [00:00<?, ?it/s]/Users/jialunlyu/Documents/GitHub/ProjectBuffalo/buffalo/ingestion/advantage.py:73: UserWarning: Ingestion results from https://www.alphavantage.co/query?function=LISTING_STATUS&date=2022-01-01&state=active&apikey=2rrNROO0beX90lPH7ixQOp0mT_9SwF0d in 0 rows.
  warnings.warn(f'Ingestion results from {url} in 0 rows.')
100%|██████████| 1/1 [00:00<00:00,  5.83it/s]


#### Economic Indicators

In [15]:
economy = pd.DataFrame({
    'function': ['FEDERAL_FUNDS_RATE', 'TREASURY_YIELD', 'NONFARM_PAYROLL', 'CPI', 'UNEMPLOYMENT'],
    'interval': ['daily', 'daily', 'monthly', 'monthly', 'monthly'],
    'maturity': '3month',
    'data_type': ingestion.enum.DataType.ECON,
    'ingestion_type': ingestion.enum.IngestionType.REST,
    'data_name': 'ECONOMY'
})

In [16]:
ingestor.ingestion_from_web(economy)

100%|██████████| 5/5 [01:01<00:00, 12.24s/it]


#### SMA Trend Indicator

In [17]:
sma_trend = pd.DataFrame({
    'symbol': tickers,
    'function': 'SMA',
    'interval': 'daily',
    'data_type': ingestion.enum.DataType.TREND_INDICATOR,
    'ingestion_type': ingestion.enum.IngestionType.REST,
    'data_name': 'SMA'
})

In [18]:
sma_trend = sma_trend.merge(pd.DataFrame({
    'time_period': [10, 50, 200]
}), how='cross').merge(pd.DataFrame({
    'series_type': ['open', 'close', 'high', 'low']
}), how='cross')

In [19]:
ingestor.ingestion_from_web(sma_trend)

100%|██████████| 288/288 [49:37<00:00, 10.34s/it]  


#### Cyclic Indicators

In [20]:
ht_sine_cycle = pd.DataFrame({
    'symbol': tickers,
    'function': 'HT_SINE',
    'interval': 'daily',
    'data_type': ingestion.enum.DataType.CYCLE_INDICATOR,
    'ingestion_type': ingestion.enum.IngestionType.REST,
    'data_name': 'HT_SINE'
})

In [21]:
ht_sine_cycle = ht_sine_cycle.merge(pd.DataFrame({'time_period': [50, 200]}), how='cross').merge(pd.DataFrame({'series_type': ['open', 'high', 'low', 'close']}), how='cross')

In [22]:
ingestor.ingestion_from_web(ht_sine_cycle)

100%|██████████| 192/192 [32:30<00:00, 10.16s/it] 


#### Momentum Indicator

In [23]:
momentum_cycle = pd.DataFrame({
    'symbol': tickers,
    'interval': 'daily',
    'data_type': ingestion.enum.DataType.CYCLE_INDICATOR,
    'ingestion_type': ingestion.enum.IngestionType.REST
})

In [24]:
momentum_cycle = momentum_cycle.merge(pd.DataFrame({'function': ['MOM', 'ROC']}), how='cross').merge(pd.DataFrame({'time_period': [50, 200]}), how='cross').merge(pd.DataFrame({'series_type': ['open', 'high', 'low', 'close']}), how='cross')

In [25]:
ingestor.ingestion_from_web(momentum_cycle)

100%|██████████| 384/384 [1:07:21<00:00, 10.52s/it]


#### Market News

In [26]:
news = pd.DataFrame({
    'symbol': tickers,
    'sort': 'Latest',
    'limit': 50,
    'data_type': ingestion.enum.DataType.MARKET_NEWS,
    'ingestion_type': ingestion.enum.IngestionType.REST,
    'data_name': 'MARKET_NEWS'
})

In [27]:
ingestor.ingestion_from_web(news)

  0%|          | 0/24 [00:00<?, ?it/s]


TypeError: market_news_sentiment_download() got an unexpected keyword argument 'symbol'

#### Storing All the Data to File

In [29]:
ingestor.data.keys()

dict_keys(['ADJUSTED_DAILY_STOCK', 'COMPANY_OVERVIEW', 'COMPANY_INCOME_STATEMENT', 'COMPANY_BALANCE_SHEET', 'COMPANY_CASH_FLOW', 'COMPANY_EARNINGS', 'COMPANY_EARNINGS_CALENDAR_3M', 'LISTING_INFO', 'ECONOMY', 'SMA', 'HT_SINE', 'NewFile', 'NewFile1'])

In [32]:
ingestor.store_data(r'cached_data/ingestion.sqlite', data_names=['COMPANY_EARNINGS', 'COMPANY_EARNINGS_CALENDAR_3M', 'LISTING_INFO', 'ECONOMY', 'SMA', 'HT_SINE'])